# Brussels Airlines Special Flights during Disruption

In [76]:
from json import loads, dumps
import pandas as pd
import numpy as np
# EXTRACT
with open("all-flights.json") as f1:
    sched_flights = pd.DataFrame(loads(f1.read())["AirlineFlightSchedulesResult"]["data"])

with open("historical_flights.json") as f2:
    hist_flights = pd.DataFrame(loads(f2.read())).T

with open("airports-new.json") as f3:
    airports = loads(f3.read())
    ap = pd.DataFrame(airports).T

airports is a python dict, which is easier for cycling through all airports and looking up individual airports by ICAO code. The Pandas dataframe is easier for joining it with other dataframes.

In [77]:
# TRANSFORM
import arrow

sched_flights['arrowdate'] = sched_flights.departuretime.apply(arrow.get)
sched_flights['date'] = sched_flights.arrowdate.apply(lambda x: x.format('YYYY-MM-DD'))
bel_sched_flights = sched_flights[sched_flights["actual_ident"]==""]
bel_sched_flights = bel_sched_flights[["ident","aircrafttype", "origin", "destination", \
                           "seats_cabin_coach", "date", "departuretime"]]

We'll perform a transformation to determine the city pairs like this. Similarly, we save the seat capacity per aircraft type.

In [78]:
citypairs = bel_sched_flights[["origin", "destination"]].drop_duplicates()

In [79]:
ac = bel_sched_flights.groupby(["aircrafttype"])
aircraft = ac.aggregate(np.mean)['seats_cabin_coach']

Let's calculate the distances for the city pairs

In [80]:
import numpy as np
from numpy import pi
def distance(lat1, lon1, lat2, lon2):
    # http://www.movable-type.co.uk/scripts/latlong.html
    R = 6371 / 1.6 # in status miles
    lat1 *= pi/180.
    lon1 *= pi/180.
    lat2 *= pi/180.
    lon2 *= pi/180.
    return R*np.arccos(
        np.sin(lat1)*np.sin(lat2) + 
        np.cos(lat1)*np.cos(lat2)*np.cos(lon2-lon1))

def airdistance(origin = "EBBR", destination = "EBBR"):
    if origin in airports and destination in airports:
        lat1 = airports[origin]["latitude"]
        lat2 = airports[destination]["latitude"]
        lon1 = airports[origin]["longitude"]
        lon2 = airports[destination]["longitude"]
        return int(distance(lat1, lon1, lat2, lon2))
    else:
        print("Not all airports known:", origin, "/",destination)
        return 0

def airdist(citypair):
    return airdistance(citypair["origin"], citypair["destination"])

There are some flights with incomplete origin info. The origin is passed as coordinates which we need to parse in a rudimentary way. 

In [81]:

def extract_coord(s):
    elem = s.split()
    t = (elem[1], elem[2]) # a tuple with two elements
    return t

def find_nearest(c):
    dist = 10000 # arbitrary high value
    airp = ""
    for a, aa in airports.items():
        d = distance(aa['latitude'], aa['longitude'], float(c[0]), float(c[1]))
        if d < dist:
            dist = d
            airp = a
    return airp

def validate_origin(o):
    if len(o)==4:    # rudimentary, but ICAO codes are 4 digits, good enough
        return o
    else:
        return find_nearest(extract_coord(o))

hist_flights['origin'] = hist_flights['origin'].apply(validate_origin)

def validate_city(city, icao):
    if not city:
        if icao in airports:
            city = airports[icao]['name'] # note that location is empty in the airports db, too
            print(city, icao)
    return city

hist_flights["originCity"] = hist_flights.apply(lambda r: validate_city(r['originCity'], 
                                                                       r['origin']), axis=1)
hist_flights["destinationCity"] = hist_flights.apply(lambda r: validate_city(r['destinationCity'], 
                                                                       r['destination']), axis=1)


Lome DXXX
Lome DXXX
Lome DXXX
N'Djili Int'l FZAA
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX
Lome DXXX


As we will see, emergency flights were flown out of Liege, Antwerp, Frankfurt, and Zurich. In order to compare the emergency flights to the ordinary flights, we need to compare the "unique" destination, so precisely the other end of the citypair. For example, a flight DAKAR-FRANKFURT will count as a DAKAR flight.

In [82]:
origin = set(["EBBR", "EBLG", "EBAW", "EDDF", "LSZH"]) # special airports considered homebase
def decide_destination(flight):
    A = set([flight["origin"], flight["destination"]])
    B = set(["EBBR"]) # without the square brackets the set would contain E, B, R
    if A <= origin:   # is A wholly contained in list if 5?
        # then substract EBBR from it
        return A.difference(B).pop() # easy way to convert set of one element to element
    else: # at least one was not in the list, so remove the list to find it.
        # if both were in the list, select one arbitrarily
        return A.difference(origin).pop() # potentially one arbitrary of two elements
    
test_flight = {"origin": "LSZH", "destination" : "KIFR"}
decide_destination(test_flight)
    

'KIFR'

Apply the function defined above to both scheduled and historical flights.

In [83]:
hist_flights["udest"] = hist_flights.apply(decide_destination, axis=1)
bel_sched_flights["udest"] = bel_sched_flights.apply(decide_destination, axis=1)

In [85]:
hist_flights.loc[hist_flights['udest']=="GUCY"]

,ADT,Filed,actualarrivaltime,actualdeparturetime,aircrafttype,destination,destinationCity,destinationName,diverted,estimatedarrivaltime,...,filed_altitude,filed_departuretime,filed_ete,filed_time,ident,origin,originCity,originName,route,udest
BEL1205-1459530459-airline-0098:0,2016-04-03 13:58,2016-04-02 15:38,1459711980,1459688317,A333,GUCY,Conakry,Conakry Int'l (Gbessia Int'l),,1459711980,...,0,1459682100,06:00:00,1459607927,BEL1205,EDDF,Frankfurt am Main,Frankfurt Int'l,,GUCY


In [86]:
citypairs["dist"] = 0
citypairs["dist"] = citypairs.apply(airdist, axis=1)


In [87]:
def available_seat_miles(row):
    origin = row['origin']
    destination = row['destination']
    seats = row['seats_cabin_coach']
    dist = airdistance(origin, destination)
    if not dist:
        print("WARNING: no dist for ", origin, " ", destination)
    return  dist * seats

bel_sched_flights["a_s_m"] = bel_sched_flights.apply(available_seat_miles, axis=1)


In [88]:
bel_sched_flights[bel_sched_flights['udest']=="GUCY"]

,ident,aircrafttype,origin,destination,seats_cabin_coach,date,departuretime,udest,a_s_m


We will now define the relevant time period as 29 March to 5 April (included).

In [66]:
t1 = arrow.get(2016, 3, 29).timestamp
t2 = arrow.get(2016, 4, 6).timestamp

In [67]:
BSF = bel_sched_flights.loc[(bel_sched_flights['departuretime'] > t1) & 
                      (bel_sched_flights['departuretime'] < t2)]
np.floor(BSF.a_s_m.sum() / 1e6)

225.0

The total available seat miles for all city pairs was 225 mn.

### Top 20 Destinations by Available Seat Miles
We can now print the aggregate seat capacity (a_s_m) by city pair.

In [68]:
g1 = BSF.groupby(['udest'])

def myroundsum(x):
    return np.round(np.sum(x)/1e6, decimals=1)

sched_agg = g1.aggregate(myroundsum)
sched_agg = sched_agg.combine_first(ap).sort_values(by="a_s_m", ascending=False)
sched_agg.ix[0:19, [0, 3, 5]]

,a_s_m,location,name
KJFK,14.9,"New York, NY",John F Kennedy Intl
FKKD,13.7,Douala,Douala Int'l
GOOY,12.4,Yoff,Dakar
KIAD,11.1,"Washington, DC",Washington Dulles Intl
CYYZ,10.9,"Toronto, Ontario",Toronto Pearson Int'l
FZAA,9.8,Kinshasa,N'Djili Int'l
LLBG,9.0,Tel Aviv,Ben Gurion Int'l
DIAP,8.2,Abidjan,Port Bouet (Felix Houphouet Boigny Int'l)
HRYR,6.5,Kigali,Kigali Int'l
DGAA,6.4,Accra,Kotoka Int'l


Let's now take a look at the historical flights that were actually flown during the same time (29/3 to 5/4).

Merge the flight info with the aircraft info to obtain the number of seats per flight.
Subset the data to only include the date range we defined above.

In [69]:
hist_flights = hist_flights.join(aircraft, on="aircrafttype")

BHF = hist_flights.loc[(hist_flights['actualdeparturetime'] > t1) & 
                      (hist_flights['actualdeparturetime'] < t2), \
        ['ident','udest','ADT','origin','destination', 'originCity', 'destinationCity', 'seats_cabin_coach']]

In [70]:
def seat_miles_flown(row):
    origin = row['origin']
    destination = row['destination']
    seats = row['seats_cabin_coach']
    dist = airdistance(origin, destination)
    if not dist:
        print("WARNING: no dist for ", origin, " ", destination)
    return  dist * seats

BHF["smf"] = BHF.apply(seat_miles_flown, axis=1)
"Total seat miles flown in millions: {}".format(np.floor(BHF.smf.sum() / 1e6))

'Total seat miles flown in millions: 54.0'

There is one flight which appears twice in the data set (most likely an error):

In [71]:
bhf_g = BHF.groupby(["ident", "udest", "ADT"])
bhf_c = bhf_g.count()['smf']
bhf_c[bhf_c>1]

ident    udest  ADT             
BEL1205  GOOY   2016-04-03 21:50    2
Name: smf, dtype: int64

### Top 20 Destinations for Brussels Emergency Flights (by seat miles flown)

In [72]:

hist_agg = BHF.groupby(['udest']).aggregate(np.sum)
hist_agg.smf = hist_agg.smf.apply(lambda x: np.round(x / 1e6, decimals=1))
hist_agg.combine_first(ap).sort_values(by="smf", ascending=False).ix[0:19, [1, 3, 5]]

,location,name,smf
KJFK,"New York, NY",John F Kennedy Intl,12.8
FZAA,Kinshasa,N'Djili Int'l,6.1
FKYS,Yaounde,Yaounde Nsimalen Int'l,3.1
HRYR,Kigali,Kigali Int'l,2.7
GOOY,Yoff,Dakar,2.4
GFLL,Freetown,Lungi Int'l,2.3
DIAP,Abidjan,Port Bouet (Felix Houphouet Boigny Int'l),2.1
GLRB,Monrovia,Roberts Int'l,1.8
DXXX,,Lome,1.6
LPPT,Lisbon,"Lisbon / Lisboa,Portela (Lisbon)",1.2


This routine shows how to obtain airport info from the flightaware API. Scheduled and historical flights were obtained in a similar fashion.

In [73]:
import requests
def AirportInfo(icao):
    username = 'REMOVED'
    apiKey = "REMOVED"
    baseURL = "http://flightxml.flightaware.com/json/FlightXML2/"
    method = "AirportInfo"
    data = {"airportCode" : icao}
    r = requests.post(baseURL+method, auth=(username, apiKey), data=data)
    if r.status_code == 200:
        res = r.json()
        return res['AirportInfoResult']
    else:
        print("Problem with airport:", icao)

AirportInfo("EGCC")

Problem with airport: EGCC


#### Comparison of Schedules (Regular versus Emergency)
We calculated the ratio of Seat Miles Flown / Available Seat Miles. The two top value reveals a data problem which we can explain by the fact that Lomé is normally only reached after a stop (so contributes little to ASM), but was directly flown to/from Frankfurt during the emergency schedule. Conakry, on the other hand, was not on the schedule, but for some reason was a stop during the emergeny schedule.

In [89]:
combo = sched_agg.combine_first(hist_agg)[['a_s_m', 'smf']]
combo["ratio"] = combo['smf']/combo['a_s_m']


It's interesting that Brussels kept flights to New York and achieved more than 85% of the scheduled capacity, but ditched Washington completely. More analysis would be required to explain this.

In [90]:
select = pd.notnull(combo["ratio"])
combo.combine_first(ap)[["name", "a_s_m", "smf", "ratio"]].sort_values(by="ratio", ascending=False)

,name,a_s_m,smf,ratio
DXXX,Lome,0.2,1.6,8.000000
LIMF,Turin Int'l (Torino Caselle),0.1,0.1,1.000000
GBYD,Banjul Int'l (Yundum Int'l),0.1,0.1,1.000000
KJFK,John F Kennedy Intl,14.9,12.8,0.859060
FZAA,N'Djili Int'l,9.8,6.1,0.622449
GFLL,Lungi Int'l,3.9,2.3,0.589744
GCLP,Gran Canaria Int'l,1.6,0.8,0.500000
LGIR,"Heraklion Int'l, Nikos Kazantzakis",0.4,0.2,0.500000
LIPE,Bologna (Guglielmo Marconi),0.6,0.3,0.500000
HRYR,Kigali Int'l,6.5,2.7,0.415385


#### Auxiliary Function to Investigate Flights to Certain Airports

In [99]:
def investigate(udest):
    print("--- SCHEDULED FLIGHTS ---")
    print(BSF.loc[BSF['udest'] == udest, ["date","ident", "origin", "destination"]])
    print("--- HISTORICAL FLIGHTS ---")
    print(BHF.sort_values(by="ADT").loc[BHF['udest'] == udest, ["ADT","ident", "origin", "destination"]])
    
investigate("GOOY")

--- SCHEDULED FLIGHTS ---
             date   ident origin destination
7043   2016-03-29  BEL206   GOOY        EBBR
7646   2016-03-29  BEL205   EBBR        GOOY
8210   2016-03-30  BEL203   EBBR        GOOY
8767   2016-03-30  BEL204   GOOY        EBBR
9327   2016-03-31  BEL205   EBBR        GOOY
9894   2016-03-31  BEL206   GOOY        EBBR
10043  2016-04-01  BEL204   GOOY        EBBR
11008  2016-04-01  BEL203   EBBR        GOOY
11213  2016-04-02  BEL203   EBBR        GOOY
11695  2016-04-02  BEL204   GOOY        EBBR
12066  2016-04-03  BEL205   EBBR        GOOY
12321  2016-04-03  BEL206   GOOY        EBBR
12902  2016-04-04  BEL203   EBBR        GOOY
13359  2016-04-04  BEL204   GOOY        EBBR
14013  2016-04-05  BEL205   EBBR        GOOY
14919  2016-04-05  BEL206   GOOY        EBBR
--- HISTORICAL FLIGHTS ---
                                                ADT    ident origin  \
BEL1204-1459241348-airline-0005:0  2016-03-31 00:35  BEL1204   GOOY   
BEL1203-1459515467-ed-0004:0       2016-

Let's use the k-means algorithm to categorize the airports into 3 continents, only using longitude and latitude. 

In [50]:
ap = pd.DataFrame(airports).T


In [109]:
import sklearn.cluster
k = sklearn.cluster.KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=300, tol=0.0001, 
                           precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1)
df = ap[["latitude", "longitude"]]
df
ki = k.fit_predict(df)
ap["continent"] = ki

Continent 1 is Africa (but not Morocco)!

In [110]:
ap.loc[ap.continent==1]

,latitude,location,longitude,name,timezone,continent
DBBB,6.35723,Cotonou,2.38435,Cadjehoun (Cotonou),:Africa/Porto-Novo,1
DFFD,12.3532,Ouagadougou,-1.51242,Ouagadougou,:Africa/Ouagadougou,1
DGAA,5.60519,Accra,-0.166786,Kotoka Int'l,:Africa/Accra,1
DIAP,5.26139,Abidjan,-3.92629,Port Bouet (Felix Houphouet Boigny Int'l),:Africa/Abidjan,1
DXXX,6.16561,,1.25451,Lome,:Africa/Lome,1
FKKD,4.00608,Douala,9.71948,Douala Int'l,:Africa/Douala,1
FKYS,3.72256,Yaounde,11.5533,Yaounde Nsimalen Int'l,:Africa/Douala,1
FNLU,-8.85838,Luanda,13.2312,Quatro de Fevereiro,:Africa/Luanda,1
FZAA,-4.38575,Kinshasa,15.4446,N'Djili Int'l,:Africa/Kinshasa,1
GBYD,13.338,Banjul,-16.6522,Banjul Int'l (Yundum Int'l),:Africa/Banjul,1


Continent 0 is Europe, Israel and Morocco.

In [111]:
ap.loc[ap.continent==0]

,latitude,location,longitude,name,timezone,continent
EBAW,51.1894,Antwerp / Deurne,4.46028,Antwerp Int'l,:Europe/Brussels,0
EBBR,50.9014,Brussels,4.48444,Brussels,:Europe/Brussels,0
EBLG,50.6374,Liege,5.44322,Liege,:Europe/Brussels,0
EDDF,50.0264,Frankfurt am Main,8.54312,Frankfurt Int'l,:Europe/Berlin,0
EDDH,53.6304,Hamburg,9.98823,Hamburg,:Europe/Berlin,0
EDDT,52.5597,Berlin,13.2877,Tegel Int'l,:Europe/Berlin,0
EGAC,54.6181,"Belfast, Northern Ireland",-5.8725,George Best Belfast City,:Europe/London,0
EGBB,52.4539,"Birmingham, England",-1.74803,Birmingham Int'l,:Europe/London,0
EGCC,53.3537,"Manchester, England",-2.27495,Manchester,:Europe/London,0
EGLL,51.4775,"London, England",-0.461389,London Heathrow,:Europe/London,0


Determine which airports are closest to the cluster centers defined by k-means:

In [112]:
for kc in k.cluster_centers_:
    print(find_nearest(kc))

LIMC
DBBB
KIAD


The result was Milan, Cotonou, and Washington. Milan as a cluster center is interesting, and the map shows how Milan sits in the middle of the destinations in Europe. (The picture is from gcmap.com)
![Map](Milan-Center.png)

Can we use k-means clustering to find out something interesting about the emergency flights? So far, not. This will remain for another day...

In [113]:
import sklearn.cluster
co = combo.fillna(0)
k = sklearn.cluster.KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=300, tol=0.0001, 
                           precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1)
ki = k.fit_predict(co)
ki

array([1, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 1,
       2, 0, 1, 2, 2, 0, 0, 2, 2, 1, 2, 2, 0, 0, 1, 1, 2, 2, 2, 0, 0, 0, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 1,
       0, 2, 0, 0, 0, 2, 2, 2], dtype=int32)

In [114]:
combo["category"] = ki
combo

,ask,smf,ratio,category
udest,,,,
CYYZ,10907976.0,NaN,NaN,1
DBBB,NaN,7.762240e+05,NaN,2
DFFD,4510278.0,9.443720e+05,0.209382,0
DGAA,6632448.0,8.457520e+05,0.127517,0
DIAP,7814310.0,1.803254e+06,0.230763,0
DXXX,NaN,1.501576e+06,NaN,2
EDDF,NaN,5.216400e+04,NaN,2
EDDH,1247043.0,NaN,NaN,2
EDDT,3316420.0,4.916580e+05,0.148250,0
